# OpenAI Part

In [1]:
import openai 
import proj_db 
import json 

# Your OpenAI api key must be provided. 

Writing the function. 

In [2]:
def get_answer(question): 
    messages = [{'role':'user', 'content':question}]
    functions = [
        {
            'name':'get_player_general_stats',
            'description':'''This function is used to get some general stats of a speicfic soccer player. It returns a dictionary with some parameters,
            all of them referred to the 2022/2023 soccer season. The parameters are: Goals, Assists, Expected Goals (how many goals he should have scored),
            Non-penalty expected goals (expected goals without penalty kicks), Progressive passes (passes made near the opponent penalty area), 
            Progressive carries (carries that move the ball towards the opponent's goal line at least 10 yards from its furthest point in the last six passes), 
            Shots on target, Goals per shot (GSh), Goals per shot on target (GSot)''',
            'parameters':{
                'type':'object',
            'properties':{
                'player_name':{
                    'type':'string',
                    'description':"Player's whom stats are needed. His first name and last name will be provided"
                }
            },
            'required':['player_name']
            }
        }
    ]
    response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = messages, 
    functions = functions,
    function_call = 'auto') 
    response_message = response['choices'][0]['message']
    if response_message.get('function_call'):
        available_functions = {'get_player_general_stats':proj_db.get_player_general_stats}
        response_message = response['choices'][0]['message'] 
        function_name = response_message['function_call']['name']
        function_to_call = available_functions[function_name]
        function_args = json.loads(response_message ['function_call']['arguments'])
        function_response = function_to_call(function_args)
        messages.append(response_message)
        messages.append({
            'role':'function',
            'name':function_name,
            'content':str(function_response)
        })

        second_response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=messages
        )
        return second_response['choices'][0]['message']['content']
    else:
        return response_message['content']

Testing.

In [24]:
if __name__ == '__main__':
    print(get_answer('What is the sum of goals and assits of Victor Osimhen?'))

The sum of goals and assists of Victor Osimhen is 30.


In [32]:
if __name__ == '__main__':
    print(get_answer('Provide me a summary of the Lazar Samardzic shooting stats.'))

Lazar Samardzic has scored a total of 5 goals and provided 4 assists. He has an expected goals (xG) of 2.4 and a non-penalty expected goals (npxG) of 2.4. Samardzic has made 100 progressive passes and completed 34 progressive carries. He has hit the target with 18 shots, resulting in a goal conversion rate of 0.1 and a shots on target conversion rate of 0.28.
